# Virtual Linking Bids for Market-Clearing with Non-Merchant Storage

This is the code for the paper "Virtual Linking Bids for Market-Clearing with Non-Merchant Storage". Different types of market clearing setups can be run and compared, including the examples presented in the paper. The code was written for Julia v1.8.

In [1]:
# Include the relevant packages
using CSV, DataFrames
using HiGHS

The different parameters can be updated in the csv files "data_gen.csv" and "data_load.csv" and below:

In [13]:
## MARKET DATA
# Duration of time periods (in hours):
∆t = 1

## STORAGE DATA
# Storage capacity (MWh)
E = 2.5 
# Initial state of energy (MWh)
E_0 = 0.0 
;

In [4]:
# Dataframes are created from the csv files for generators:
G_df = CSV.read("data_gen.csv", DataFrame)

Row,MI,t,ID,max,cost
,Int64,Int64,Int64,Int64,Int64
1,1,1,1,2,5
2,1,1,2,2,10
3,2,1,1,2,2
4,2,1,2,2,9


In [5]:
# And loads:
L_df = CSV.read("data_load.csv", DataFrame)

Row,MI,t,ID,max,utility
,Int64,Int64,Int64,Int64,Int64
1,1,1,1,0,12
2,2,1,1,3,12


## Ideal Market Clearing

In [ ]:
# Set the level for the storage system at the end of the test period:
E_final = 0.0
;

In [16]:
# Run the market clearing
include.(("fct_perfect.jl","fct_mc_all.jl","fct_data_all.jl"))
df_MCP = perfect(∆t, E, E_0, E_final, G_df, L_df)

SW_tot: 27.0
surplus_gen_tot: [6.0, 0.0]
surplus_gen_sum_tot: 6.0
surplus_load_tot: [21.0]
surplus_stg_tot: 0.0


Row,Time,Market_Price,C1,p1,P1,C2,p2,P2,U1,d1,D1,p_C,e
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,1.0,5.0,5.0,1.0,2.0,10.0,0.0,2.0,12.0,-0.0,0.0,1.0,1.0
2,2.0,5.0,2.0,2.0,2.0,9.0,0.0,2.0,12.0,3.0,3.0,-1.0,0.0


## Split Market Clearing with Level

In [18]:
# Set the level for the storage system at the end of each market interval:
E_end = [1.0, 0.0]
;

In [25]:
# Run the market clearing
include.(("fct_split.jl","fct_mc_all.jl","fct_data_all.jl"))
S_end = zeros(maximum(G_df.MI)) # Set the penalty to zero (no penalty)
df_MCS = split(∆t, E, E_0, G_df, L_df, E_end, S_end)

SW_tot: 27.0
surplus_gen_tot: [0.0, 0.0]
surplus_gen_sum_tot: 0.0
surplus_load_tot: [30.0]
surplus_stg_tot: -3.0


Row,Time,Market_Price,C1,p1,P1,C2,p2,P2,U1,d1,D1,p_C,e
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,1.0,5.0,5.0,1.0,2.0,10.0,0.0,2.0,12.0,-0.0,0.0,1.0,1.0
2,2.0,2.0,2.0,2.0,2.0,9.0,0.0,2.0,12.0,3.0,3.0,-1.0,0.0


## Split Market Clearing with Penalty

In [26]:
# Set the penalty for the storage system at the end of each market interval:
S_end = [2, 0]
;

In [27]:
# Run the market clearing
include.(("fct_split.jl","fct_mc_all.jl","fct_data_all.jl"))
E_end = zeros(maximum(G_df.MI)) # Set the penalty to zero (no penalty)
df_MCS = split(∆t, E, E_0, G_df, L_df, E_end, S_end)

SW_tot: 23.0
surplus_gen_tot: [14.0, 0.0]
surplus_gen_sum_tot: 14.0
surplus_load_tot: [9.0]
surplus_stg_tot: 0.0


Row,Time,Market_Price,C1,p1,P1,C2,p2,P2,U1,d1,D1,p_C,e
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,1.0,0.0,5.0,0.0,2.0,10.0,0.0,2.0,12.0,-0.0,0.0,-0.0,0.0
2,2.0,9.0,2.0,2.0,2.0,9.0,1.0,2.0,12.0,3.0,3.0,-0.0,0.0


## Market Clearing with Virtual Linking Bids

In [32]:
# Set the level for the storage system at the end of each market interval:
E_end = [1.0, 0.0]
# Set a discount between 0 and 1 for the value of the stored energy (0: no discount)
discount = 0
;

In [34]:
# Version with solution multiplicity
include.(("fct_vlb.jl","fct_mc_vlb.jl","fct_stor_val_update.jl","fct_data_all.jl"))
df_VLB = VLB_no_bin(∆t, E, G_df, L_df, E_end, discount)

SW_tot: 27.0
surplus_gen_tot: [6.0, 0.0]
surplus_gen_sum_tot: 6.0
surplus_load_tot: [21.0]
surplus_stg_tot: 0.0

S: [0.0]
E_init: [0.0]


Row,Time,Market_Price,C1,p1,P1,C2,p2,P2,U1,d1,D1,p_C_intra,e_intra,p_D_inter,e_inter
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,1.0,5.0,5.0,1.0,2.0,10.0,0.0,2.0,12.0,-0.0,0.0,1.0,1.0,-0.0,0.0
2,2.0,5.0,2.0,2.0,2.0,9.0,0.0,2.0,12.0,3.0,3.0,-0.0,-0.0,1.0,-0.0


In [39]:
# Version without solution multiplicity (binary variables)
include.(("fct_vlb_bin.jl","fct_mc_vlb_bin.jl","fct_mc_vlb_no_bin.jl","fct_stor_val_update.jl","fct_data_all.jl"))
df_VLB_bin = VLB_bin(∆t, E, G_df, L_df, E_end, discount)

SW_tot: 27.0
surplus_gen_tot: [6.0, 0.0]
surplus_gen_sum_tot: 6.0
surplus_load_tot: [21.0]
surplus_stg_tot: 0.0

In the inter-storage at the end of the test period: 
S: [0.0]
E_init: [0.0]


Row,Time,Market_Price,C1,p1,P1,C2,p2,P2,U1,d1,D1,p_C_intra,e_intra,p_D_inter,e_inter
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,1.0,5.0,5.0,1.0,2.0,10.0,0.0,2.0,12.0,-0.0,0.0,1.0,1.0,-0.0,0.0
2,2.0,5.0,2.0,2.0,2.0,9.0,0.0,2.0,12.0,3.0,3.0,-0.0,-0.0,1.0,-0.0
